In [61]:
import pandas as pd
import plotly.graph_objs as go

In [62]:
vm_sizes = [1, 2, 4, 8, 16, 20]
mig_methods = ['Vanilla_PreCopy', 'Vanilla_PostCopy', 'XBZRLE_Enabled_PreCopy']

In [63]:
from tkinter import font


def mig_time_plot(plot_column="Total-Time", y_title="Total Migration Time (s)"):
    mig_data = {}
    for mig_method in mig_methods:
        mig_data[mig_method] = {"data": [], "deviation": []}
        for vm_size in vm_sizes:
            vm_size_mb = str(vm_size*1024)
            data = pd.read_csv(f'./data/{mig_method}/{vm_size_mb}.csv')
            mig_data[mig_method]["data"].append(data[plot_column].mean()/1000)
            mig_data[mig_method]["deviation"].append(data[plot_column].std()/1000)

    data = pd.DataFrame({'VM size (GB)': [ str(vm_size) for vm_size in vm_sizes],
                        'Vanilla_PreCopy': mig_data['Vanilla_PreCopy']["data"],
                        'Vanilla_PostCopy': mig_data['Vanilla_PostCopy']["data"],
                        'XBZRLE_Enabled_PreCopy': mig_data['XBZRLE_Enabled_PreCopy']["data"]})
    trace1 = go.Bar(x=data['VM size (GB)'], y=data['Vanilla_PostCopy'], name='Vanilla Post-copy', error_y=dict(type='data', array=mig_data['Vanilla_PostCopy']["deviation"]))
    trace2 = go.Bar(x=data['VM size (GB)'], y=data['Vanilla_PreCopy'], name='Vanilla Pre-copy', error_y=dict(type='data', array=mig_data['Vanilla_PreCopy']["deviation"]))
    trace3 = go.Bar(x=data['VM size (GB)'], y=data['XBZRLE_Enabled_PreCopy'], name='Pre-copy w/ XBZRLE', error_y=dict(type='data', array=mig_data['XBZRLE_Enabled_PreCopy']["deviation"]))

    layout = go.Layout(
                    xaxis=dict(title='VM size (GB)'),
                    yaxis=dict(title=y_title),
                    barmode='group',
                    template='simple_white'
                    )

    fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

    fig.update_layout(
        width=677,  # 0.45 textwidth
        height=338,
        font=dict(family="Arial", size=17.2, color="black"),
        margin=dict(t=15, r=5, l=70, b=70, autoexpand=True),
        yaxis_title_standoff=10,
        xaxis_title_standoff=8,
        legend=dict(
            font=dict(family="Arial", size=19, color="black"),
            orientation="h",
            yanchor="bottom",
            y=-0.45,
        ),
    )
    fig.write_image(f"./VM-size-vs-{plot_column}.png",scale=4, width=677, height=338)

    fig.show()

In [64]:
mig_time_plot()

In [65]:
mig_time_plot("Downtime", "Downtime (s)")